In [584]:
# look for sequence inserted in transMap of AugustusPB back to human
from tools.psl import *
from tools.transcripts import *
import os
import pandas as pd
from cat.consensus import *
from collections import *
from tools.fileOps import *
from tools.sqlInterface import *
from tools.nameConversions import strip_alignment_numbers
data_dir = '/public/groups/cgl/cat/primates_evan/rhesus_reverse/work/transMap'

In [38]:
alns = get_alignment_dict(os.path.join(data_dir, 'Human.filtered.psl'))

In [33]:
txs = get_gene_pred_dict(os.path.join(data_dir, 'Human.filtered.gp'))

In [212]:
ref_txs = get_gene_pred_dict('/public/groups/cgl/cat/primates_evan/rhesus_reverse/work/reference/Rhesus.gp')

In [113]:
ref_df = load_annotation('/public/groups/cgl/cat/primates_evan/rhesus_reverse/out/databases/Rhesus.db')

In [ ]:
# filter for 1-1 mappings
unique = set(tm_eval_df[tm_eval_df.UnfilteredParalogy == ''].TranscriptId)

In [199]:
# filter for novel *isoform* predictions
novel = set(ref_df[(ref_df.TranscriptBiotype == 'unknown_likely_coding') & (ref_df.GeneBiotype != 'unknown_likely_coding')].TranscriptId)

In [200]:
useful_ids = novel & unique
print(len(useful_ids))

4951


In [397]:
def convert_ref_chrom(ref_tx, left_pos, right_pos):
    if ref_tx.strand == '+':
        rhesus_right_chrom = ref_tx.mrna_coordinate_to_chromosome(right_pos)
        rhesus_left_chrom = ref_tx.mrna_coordinate_to_chromosome(left_pos)
    else:
        rhesus_right_chrom = ref_tx.mrna_coordinate_to_chromosome(left_pos) + 1
        rhesus_left_chrom = ref_tx.mrna_coordinate_to_chromosome(right_pos - 1)
    assert rhesus_right_chrom > rhesus_left_chrom
    return rhesus_left_chrom, rhesus_right_chrom


def find_exons(ref_tx, rhesus_left_chrom, rhesus_right_chrom):
    i = ChromosomeInterval(ref_tx.chromosome, rhesus_left_chrom, rhesus_right_chrom, ref_tx.strand)
    intersections = []
    for exon in ref_tx.exon_intervals:
        if exon.overlap(i):
            intersections.append(exon)
    return intersections


def convert_back_to_tx(ref_tx, exon):
    # convert to tx coordinates
    left_pos = ref_tx.chromosome_coordinate_to_mrna(exon.start)
    right_pos = ref_tx.chromosome_coordinate_to_mrna(exon.stop - 1)
    if right_pos < left_pos:
        right_pos, left_pos = left_pos, right_pos
    return left_pos, right_pos

def find_indels(aln, tx, ref_tx, min_size=30):
    # remember where we were last iteration
    q_pos = 0
    t_pos = 0
    # iterate over block starts[i], q_starts[i + 1], t_starts[i + 1]
    for block_size, q_start, t_start in zip(*[aln.block_sizes, aln.q_starts[1:], aln.t_starts[1:]]):
        q_offset = q_start - block_size - q_pos
        t_offset = t_start - block_size - t_pos
        if q_offset != 0:  # query insertion -> insertion in target sequence
            if tx.strand == '+':
                left_pos = q_start - q_offset
                right_pos = q_start
                right_chrom = aln.query_coordinate_to_target(right_pos + 1)
                left_chrom = aln.query_coordinate_to_target(left_pos - 1)
            else:
                left_pos = aln.q_size - q_start
                right_pos = aln.q_size - q_start + q_offset
                right_chrom = aln.query_coordinate_to_target(left_pos - 1)
                left_chrom = aln.query_coordinate_to_target(right_pos + 1)
            if left_chrom is None or right_chrom is None:
                continue
            assert left_chrom < right_chrom
            assert left_pos < right_pos
            size = right_pos - left_pos
            if size >= min_size:
                rhesus_left_chrom, rhesus_right_chrom = convert_ref_chrom(ref_tx, left_pos, right_pos)
                for exon in find_exons(ref_tx, rhesus_left_chrom, rhesus_right_chrom):
                    exon_left_pos, exon_right_pos = convert_back_to_tx(ref_tx, exon)
                    yield tx.chromosome, left_chrom, right_chrom, tx.name, exon_left_pos, exon_right_pos, len(exon), exon.chromosome, exon.start, exon.stop
        q_pos = q_start
        t_pos = t_start

In [566]:
r = []
# maintain a set to keep out duplicates (augPB/augCGP predicting the same novel exon more than once)
seen_exons = set()
for aln_id, tx in txs.items():
    if strip_alignment_numbers(aln_id) not in useful_ids:
        continue
    aln = alns[aln_id]
    ref_tx = ref_txs[strip_alignment_numbers(aln_id)]
    for val in find_indels(aln, tx, ref_tx):
        i = (val[-3], val[-2], val[-1])
        if i in seen_exons:
            continue
        seen_exons.add(i)
        r.append(val)

In [567]:
rdf = pd.DataFrame(r, columns=['chromosome', 'chrom_start', 'chrom_stop', 'aln_id', 'start', 'stop', 'size',
                              'rhesus_chromosome', 'rhesus_chrom_start', 'rhesus_chrom_stop'])

In [568]:
!mkdir -p bam_tmp
rdf[['rhesus_chromosome', 'rhesus_chrom_start', 'rhesus_chrom_stop']].to_csv('bam_tmp/exons.bed', header=None, index=None, sep='\t')

In [598]:
# grab CAT consensus transcripts that are not CGP/PB to filter out false positives
rhesus_cat_df = pd.read_csv('/public/groups/cgl/cat/primates_evan/out/consensus_gene_set/Rhesus.gp_info', sep='\t')
to_keep = set(rhesus_cat_df[rhesus_cat_df.transcript_biotype != 'unknown_likely_coding'].transcript_id)
with open('bam_tmp/rhesus_cat_exons.bed', 'w') as fh:
    for tx in gene_pred_iterator('/public/groups/cgl/cat/primates_evan/out/consensus_gene_set/Rhesus.gp'):
        if tx.name in to_keep:
            for exon in tx.exon_intervals:
                fh.write('\t'.join(map(str, [tx.chromosome, exon.start, exon.stop])) + '\n')

In [599]:
!bedtools intersect -v -a bam_tmp/exons.bed -b bam_tmp/rhesus_cat_exons.bed > bam_tmp/exons_filtered_for_cat_consensus.bed

In [ ]:
# filter for IsoSeq support
!samtools merge bam_tmp/merged.bam /public/groups/cgl/cat/primates_evan/out/assemblyHub/Rhesus/*bam
!bedtools bamtobed -split -i bam_tmp/merged.bam > bam_tmp/merged.bed
!bedSort bam_tmp/merged.bed bam_tmp/merged.bed
!bedtools merge -i bam_tmp/merged.bed -c 1 -o count > bam_tmp/collapsed.bed

In [172]:
with open('bam_tmp/filtered.bed', 'w') as fh:
    for l in open('bam_tmp/collapsed.bed'):
        l = l.split()
        if int(l[-1]) > 3:
            fh.write('\t'.join(l) + '\n')

In [600]:
!bedtools intersect -u -a bam_tmp/exons.bed -b bam_tmp/collapsed.bed > bam_tmp/exons_filtered_for_isoseq.bed

In [612]:
m = rdf.copy()
for bed, col in [['bam_tmp/exons_filtered_for_cat_consensus.bed', 'cat_consensus_filter'],
                ['bam_tmp/exons_filtered_for_isoseq.bed', 'isoseq_filter']]:
    tmp_df = pd.read_csv(bed, sep='\t', header=None)
    tmp_df.columns = ['rhesus_chromosome', 'rhesus_chrom_start', 'rhesus_chrom_stop']
    tmp_df[col] = [True] * len(tmp_df)
    m = m.merge(tmp_df, on=['rhesus_chromosome', 'rhesus_chrom_start', 'rhesus_chrom_stop'], how='left').drop_duplicates()
m = m.fillna(False)
# flip the consensus to match
m['cat_consensus_filter'] = [not x for x in m['cat_consensus_filter']]

In [613]:
# add some reference info
tmp_df = ref_df[['TranscriptId', 'GeneBiotype', 'GeneId', 'GeneName']]
tmp_df.columns = ['aln_id', 'Gene Biotype', 'Gene ID', 'Gene Name']
# strip the id from m
m['aln_id'] = [strip_alignment_numbers(x) for x in m['aln_id']]

In [614]:
m = m.merge(tmp_df, on='aln_id', how='left')

In [616]:
name_map = OrderedDict([['Gene ID', 'Rhesus gene ID'],
                       ['Gene Name', 'Gene name'],
                       ['Gene Biotype', 'Gene type'],
                       ['rhesus_chromosome', 'RheMac10 chromosome'],
                       ['rhesus_chrom_start', 'RheMac10 start'],
                       ['rhesus_chrom_stop', 'RheMac10 end'],
                       ['chromosome', 'GRCh38 chromosome'],
                       ['chrom_start', 'GRCh38 nearest chromosome start'],
                       ['chrom_stop', 'GRCh38 nearest chromosome end'],
                       ['size', 'Exon size'],
                       ['cat_consensus_filter', 'Overlaps a CAT comparative transcript'],
                       ['isoseq_filter', 'Has at least 3 supporting IsoSeq reads']])
m = m.rename(columns=name_map)
m = m[name_map.values()]

In [617]:
m = m.sort_values(['Overlaps a CAT comparative transcript',
                  'RheMac10 chromosome',
                  'RheMac10 start'])

In [618]:
m.to_csv('rhesus_novel_exons.tsv', sep='\t', index=None)
m.to_excel('rhesus_novel_exons.xlsx')

In [619]:
m[['RheMac10 chromosome', 'RheMac10 start', 'RheMac10 end']].to_csv('rhesus_unfiltered_novel_exons.bed', sep='\t', header=None, index=None)
m[m['Overlaps a CAT comparative transcript'] == False][['RheMac10 chromosome', 'RheMac10 start', 'RheMac10 end']].to_csv('rhesus_filtered_novel_exons.bed', sep='\t', header=None, index=None)

In [620]:
m

,Rhesus gene ID,Gene name,Gene type,RheMac10 chromosome,RheMac10 start,RheMac10 end,GRCh38 chromosome,GRCh38 nearest chromosome start,GRCh38 nearest chromosome end,Exon size,Overlaps a CAT comparative transcript,Has at least 3 supporting IsoSeq reads
3,Rhesus_G0002154,S100A15A,processed_pseudogene,chr1,97658864,97659094,chr1,153396765,153400385,230,False,True
4,Rhesus_G0003082,HFM1,protein_coding,chr1,132859086,132859193,chr1,91277060,91324674,107,False,True
5,Rhesus_G0003082,HFM1,protein_coding,chr1,132859288,132859336,chr1,91277060,91324674,48,False,True
6,Rhesus_G0003082,HFM1,protein_coding,chr1,132863175,132863273,chr1,91277060,91324674,98,False,True
7,Rhesus_G0003082,HFM1,protein_coding,chr1,132863355,132863487,chr1,91277060,91324674,132,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
239,Rhesus_G0056391,GPM6B,protein_coding,chrX,13436359,13436425,chrX,13776257,13783364,66,True,True
240,Rhesus_G0056391,GPM6B,protein_coding,chrX,13437441,13437515,chrX,13776257,13783364,74,True,True
241,Rhesus_G0056391,GPM6B,protein_coding,chrX,13440180,13440352,chrX,13776257,13783364,172,True,True
245,Rhesus_G0058144,MAP7D3,protein_coding,chrX,132333022,132333651,chrX,136231799,136231801,629,True,True
